# Data Wrangling, Data Intuition & Wisdom of the Crowds

Putting it all together:
- pandas
- data wrangling
- data intuition
- wisdom of the crowds
- EDA

We'll use your responses to the Fermi estimation Google Form to do this.

## Setup

In [1]:
# import working with data libraries
import pandas as pd
import numpy as np

## The Data

In [2]:
# read data into Python
df = pd.read_csv('data/fermi_wi21.csv')

In [3]:
# take a look at the data
df

,Timestamp,How fast does human hair grow (cm/yr)?,"If every living person stood crammed together side-by-side, how large of an area would they occupy (km²)?","How many days would it take to walk from here to New York City (assuming no stopping to fix shoes, apply sunscreen, or for sleeping, eating, or other biological needs)?"
0,1/28/21 13:35,10 cm/1 year,"10,000",70
1,1/29/21 9:51,10,100000,100
2,1/29/21 9:51,10,100000,100
3,1/29/21 9:51,3,400,10 days
4,1/29/21 9:51,5,"1,000,000,000",30
...,...,...,...,...
78,1/31/21 14:43,25,1.2,100
79,1/31/21 18:54,100,1000,100
80,1/31/21 19:13,10,1000000,100000
81,1/31/21 20:41,5,25,4


## Data Cleaning & Wrangling

Tidy Data Rules (Review):
1. Every observation in a row
2. Every variable in a column
3. If multiple tables, column on which to merge

### iClicker Question #1

Are these data in the tidy data format?

- A) Yes, these data are ready to analyze
- B) Yes, but there is more work to do before analysis
- C) No, not tidy
- D) Have no idea what you're talking about

## Brainstorming

What considerations do we have to make about these data?

- handling non-numeric answers (i.e. "lots")
- handle units in answer
- unreasonably large/small number (outliers)? (not thoughtful, or joking)
- scientific notation

In [4]:
# change column names
df.columns = ['timestamp', 'hair_growth', 'crammed', 'SAN_NYC']
df.head()

,timestamp,hair_growth,crammed,SAN_NYC
0,1/28/21 13:35,10 cm/1 year,"10,000",70
1,1/29/21 9:51,10,100000,100
2,1/29/21 9:51,10,100000,100
3,1/29/21 9:51,3,400,10 days
4,1/29/21 9:51,5,"1,000,000,000",30


In [5]:
# check type of each Series (column)
df.dtypes

timestamp      object
hair_growth    object
crammed        object
SAN_NYC        object
dtype: object

In [6]:
# this cell will produce an error
df['hair_growth'].median()

TypeError: 

## Cleaning: Hair Growth

How fast does human hair grow (cm/yr)?

### iClicker Question #2

Which do you think is closest the actual speed at which hair grows (in cm/yr)?

- A) 1
- B) 10
- C) 100
- D) 1000

In [7]:
# take a look at unique values
df["hair_growth"].unique()

array(['10 cm/1 year', '10', '3', '5', '100', '2', '20', '0.5/1',
       '8cm/1yr', '15', '60', '30', '12cm per year', '1.25', '20 cm/yr',
       '12cm/yr', '21', '100 cm/year ', '10cm/yr', '24', '36',
       '12.7 centimeters', '6 inches a year', '12', '110',
       "6 My hair doesn't grow that much in a year", '8', '2-3cm/yr',
       '15.24', '4', '2 cm/yr', '25/1.5', '10^2cm/yr', '7', '1000',
       '60 cm/yr', '1.1', '10^2', '25', '23'], dtype=object)

In [8]:
# standardize height column
def standardize_hair(string):
    
    # Basic string pre-processing
    string = string.lower()
    string = string.strip()
    
    # take care of some extra info
    string = string.replace("my hair doesn't grow that much in a year", "")
    string = string.replace("25/1.5", "16.67")

    # wrong unit used
    # 1 in == 2.54 cm
    string = string.replace("6 inches a year", "15.24")
    
    # range provided
    string = string.replace("2-3", "2.5")
    

    # take care of inclded unit cases        
    string = string.replace("cm/year", "")
    string = string.replace("cm/ye", "")
    string = string.replace("centimeters", "")
    string = string.replace("cm per year", "")
    string = string.replace("cm/yr", "")
    string = string.replace("cm/1yr", "")
    string = string.replace("cm/1 year", "")
    string = string.replace("yr", "")
    string = string.replace("cm", "")    
    string = string.replace("/", "")

    string = string.replace("^2", "0")
    
    string = string.strip()

    # convert to numeric
    output = float(string)
    
    return output

In [9]:
# apply function across values in hair growth columns
df["hair_growth"] = df["hair_growth"].apply(standardize_hair)
df["hair_growth"].unique()

array([1.000e+01, 3.000e+00, 5.000e+00, 1.000e+02, 2.000e+00, 2.000e+01,
       5.100e-01, 8.000e+00, 1.500e+01, 6.000e+01, 3.000e+01, 1.200e+01,
       1.250e+00, 2.100e+01, 2.400e+01, 3.600e+01, 1.270e+01, 1.524e+01,
       1.100e+02, 6.000e+00, 2.500e+00, 4.000e+00, 1.667e+01, 7.000e+00,
       1.000e+03, 1.100e+00, 2.500e+01, 2.300e+01])

In [10]:
df['hair_growth'].describe()

count      83.000000
mean       38.641084
std       111.355994
min         0.510000
25%        10.000000
50%        12.700000
75%        30.000000
max      1000.000000
Name: hair_growth, dtype: float64

In [11]:
# compare to actual value: 15 cm/year (~6 in)
df["hair_growth"].median()

12.7

## Cleaning: Crammed

If every living person stood crammed together side-by-side, how large of an area would they occupy (km²)?

### iClicker Question #3

Which do you think is closest the actual area every living person would occupy (in km²)?

- A) 100
- B) 1,000
- C) 10,000
- D) 100,000

In [ ]:
df['crammed'].unique()

In [ ]:
# standardize crammed column
# code from last quarter - needs to be updated
def standardize_crammed(string):
    
    orig = string
    output = None
    
    # Basic string pre-processing
    string = string.lower()
    string = string.strip()
    
    # take care of commas
    string = string.replace(",", "")

    # take care of inclded unit cases
    string = string.replace("(km^2)", "")
    string = string.replace("km^2", "")
    string = string.replace("km²", "")
    string = string.replace("km", "")

    string = string.strip()
    
    # take care of scientific notation / word cases
    string = string.replace("x10^10", "0000000000")
    string = string.replace("7.5 x 10^6", "7500000")
    string = string.replace("x10^2", "00")
    string = string.replace(" million", "000000")
    
    # take care of case with space in middle
    string = string.replace("100 000", "100000")


    # convert to numeric
    output = float(string)
    
    return output

In [ ]:
# apply function across values in crammed columns
df["crammed"] = df["crammed"].apply(standardize_crammed)
df["crammed"].unique()

In [ ]:
# check to see how commas are being handled
df.dtypes

In [ ]:
# compare to actual value: 1,000-10,000 km^2)
df['crammed'].median()

## Cleaning: San Diego to NYC

How many days would it take to walk from here to New York City (assuming no stopping to fix shoes, apply sunscreen, or for sleeping, eating, or other biological needs)?

In [ ]:
df['SAN_NYC'].unique()

In [ ]:
# standardize distance column
# code from last quarter - needs to be updated
def standardize_distance(string):
    
    orig = string
    output = None
    
    # Basic string pre-processing
    string = string.lower()
    string = string.strip()
    
    # remove units
    string = string.replace("days", "")
    
    # take care of non-days answers
    string = string.replace("4 months", "120")
    string = string.replace("1 year", "365")
    string = string.replace("lots", "NaN")
    
    # take care of scientific notation 
    string = string.replace("10^3", "10000")
    
    string = string.strip()

    # convert to numeric
    output = float(string)
    
    return output

In [ ]:
# apply function across values in crammed columns
df["SAN_NYC"] = df["SAN_NYC"].apply(standardize_distance)
df["SAN_NYC"].unique()

In [ ]:
# compare to actual value: 38 days)
df['SAN_NYC'].median()

## Take Home Messages

1. Look at your data! (People behave unexpectedly.)
2. Get your data into a tidy data format.
3. Fix (Standardize) your data - or, better yet, collect better data!
4. Use your intuition/knowledge/information out there to consider if values make sense.